# Build an Interactive Dashboard with Plotly Dash

In this lab, we will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

<ul>
    <li> TASK 1: Add a Launch Site Drop-down Input Component </li>
    <li> TASK 2: Add a callback function to render <b>success-pie-chart</b> based on selected site dropdown </li>
    <li> TASK 3: Add a Range Slider to Select Payload </li>
    <li> TASK 4: Add a callback function to render the <b>success-payload-scatter-chart</b> scatter plot </li>
</ul>

*Note: Please take screenshots of the Dashboard and save them. Further upload your notebook to github.*

*The github url and the screenshots are later required in the presentation slides.*

After visual analysis using the dashboard, you should be able to obtain some insights to answer, for instance, the following question: which site has the highest launch success rate?

## Setup development environment

### Download a skeleton dashboard application and dataset

In [1]:
# import libraries
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

First, let's get the SpaceX Launch dataset for this lab:

In [2]:
spacex_launch_dash = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [3]:
min_payload = int(spacex_launch_dash["Payload Mass (kg)"].min())
max_payload = int(spacex_launch_dash["Payload Mass (kg)"].max())

# initialize the Dash app
app = dash.Dash(__name__)

# clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

app.layout = html.Div(
    children=[
        html.H1("SpaceX Launch Records Dashboard", style={"textAlign": "center", "color": "#503D36", "font-size": 26}),
        html.Div(
            [html.Label("All Sites:"),
             dcc.Dropdown(
                 id="site-dropdown",
                 options=[{"label": "All Sites", "value": "ALL"},
                          {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
                          {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"},
                          {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                          {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"}],
                 value="ALL",  # set a default so callback has a value at load
                 placeholder="Select a Launch Site here",
                 searchable=True)]
        ),
        html.Div(
            [html.Label("Payload range (kg):"),
             dcc.RangeSlider(
                 id="payload-slider",
                 min=0,
                 max=10000,
                 step=1000,                # change step to taste
                 marks={0: "0", 10000: "10000"},
                 value=[min_payload, max_payload])]
        ),
        dcc.Graph(id="success-pie-chart"),
        dcc.Graph(id="success-payload-scatter-chart")
    ]
)

@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"),
    Input(component_id="site-dropdown", component_property="value")
)

def get_pie_chart(entered_site):
    # handle initial None as ALL
    if entered_site is None or entered_site == "ALL":
        # sum successes by site (class == 1)
        grouped = (
            spacex_launch_dash.groupby("Launch Site")["class"].sum().reset_index()
        )  # 'class' is 0/1; summing gives count of successes
        fig = px.pie(
            grouped,
            values="class",
            names="Launch Site",
            title="Total Successful Launches by Site",
        )
        return fig
    else:
        # filter for the chosen site and count successes vs failures
        df_site = spacex_launch_dash[spacex_launch_dash["Launch Site"] == entered_site]
        outcome_counts = (
            df_site["class"].value_counts().rename_axis("outcome").reset_index(name="count")
        )
        # map 1/0 to readable labels
        outcome_counts["outcome"] = outcome_counts["outcome"].map({1: "Success", 0: "Failure"})
        fig = px.pie(
            outcome_counts,
            values="count",
            names="outcome",
            title=f"Launch Outcomes for site: {entered_site}",
        )
        return fig

@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"),
    [Input(component_id="site-dropdown", component_property="value"), Input(component_id="payload-slider", component_property="value")]
)

def render_scatter(entered_site, payload_range):
    # determine payload bounds
    if payload_range is None:
        low, high = min_payload, max_payload
    else:
        low, high = payload_range

    # filter by payload range first
    df_filtered = spacex_launch_dash[
        (spacex_launch_dash["Payload Mass (kg)"] >= low)
        & (spacex_launch_dash["Payload Mass (kg)"] <= high)
    ]
    
    # then filter by site if a specific one selected
    if entered_site is not None and entered_site != "ALL":
        df_filtered = df_filtered[df_filtered["Launch Site"] == entered_site]

    # if no data after filtering, return an empty figure with an explanatory title
    if df_filtered.empty:
        fig = px.scatter(
            title=f"No data for selected filters (site={entered_site}, payload {low}-{high} kg)"
        )
        return fig

    # Create scatter
    fig = px.scatter(
        df_filtered,
        x="Payload Mass (kg)",
        y="class",
        color="Booster Version Category" if "Booster Version Category" in df_filtered.columns else "Launch Site",
        labels={"class": "Launch Outcome (0=failure,1=success)"},
        title=(
            f"Payload vs. Launch Outcome"
            + (f" for {entered_site}" if entered_site and entered_site != "ALL" else "")
            + f"<br>Payload between {low} and {high} kg"
        ),
    )

    # make the y-axis show clearer tick labels (0 and 1)
    fig.update_yaxes(tickmode="array", tickvals=[0, 1], ticktext=["Failure", "Success"])

    return fig

if __name__ == "__main__":
    app.run()